In [4]:
from logging.handlers import RotatingFileHandler
import yahoo_fin.stock_info as si
import pandas as pd
import time

In [5]:
balance_sheet = []
income_statement = []
cfs = []
years = []
profitalyze_score = 0
tickers = si.tickers_sp500()
ROA = 0
CF = 0
change_ROA = 0
acc = 0

data_summary = pd.DataFrame(columns=['Ticker','Profitalyze', 'ROA', 'Change in ROA', 'Operating Cash Flow', 'Accruals'])

In [6]:
def get_data(stock):
    global balance_sheet
    global income_statement
    global cfs
    global years
    balance_sheet = si.get_balance_sheet(stock)
    income_statement = si.get_income_statement(stock)
    cfs = si.get_cash_flow(stock)
    years = balance_sheet.columns

In [7]:
def profitalyze():
    global profitalyze_score
    global ROA
    global CF
    global change_ROA
    global acc
    net_income = income_statement[years[0]]['netIncome']
    ROA = net_income
    net_income_prior = income_statement[years[1]]['netIncome']
    # First two scores, based on net income in the income statement (Return on Assets), roughly align with 
    # a 1 to 1.5 range within the same 1-point bracket for Return on Assets.
    net_inc_score = 1 if net_income > 0 else 0
    net_inc_score_comp = 1 if net_income > net_income_prior else 0

    # Second score calculated using Cash Flow Statement's total cash from operating activities
    operating_cf = cfs[years[0]]['totalCashFromOperatingActivities']
    CF = operating_cf
    operating_cf_score = 1 if operating_cf > 0 else 0

    # Third score is determined using net change in ROA assets
    average_assets = (balance_sheet[years[0]]['totalAssets'] + balance_sheet[years[1]]['totalAssets']) / 2
    prev_asset_average = (balance_sheet[years[1]]['totalAssets'] + balance_sheet[years[2]]['totalAssets']) / 2
    roa = net_income / average_assets
    prev_roa = net_income_prior / prev_asset_average
    change_ROA = roa - prev_roa
    print(change_ROA)
    if (roa > prev_roa ):
        roa_score = 1
    else:
        roa_score = 0

    # fourth score
    # Accruals yield 1 point if operating_cf divided by total assets is higher than 
    # ROA in current year, else 0
    total_assets = balance_sheet[years[0]]['totalAssets']
    accruals = operating_cf / total_assets
    acc = accruals
    acc_score = 1 if accruals > 0 else 0


    profitalyze_score = net_inc_score + operating_cf_score + return_on_assets_score + acc_score
    print(profitalyze_score, "/ 4")

In [ ]:
for ticker in tickers[32:76]:
    get_data(ticker)
    profitalyze()
    new_row = {'Ticker': ticker,
                'Profitalyze': profitalyze_score,
                'ROA': ROA,
                'Change in ROA': change_ROA,
                'Operating Cash Flow': CF,
                'Accruals': acc}

    data_summary = data_summary.append(new_row, ignore_index=True)
    print(ticker + ' added.')
    time.sleep(3)
data_summary.to_csv('Profitalyze_Analysis.csv')
